In [1]:
# Import our libraries
import pandas as pd
import numpy as np
# Import sklearn libraries
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve, auc, make_scorer, confusion_matrix, f1_score, fbeta_score
# Import the Naive Bayes, logistic regression, Bagging, RandomForest, AdaBoost, GradientBoost, Decision Trees and SVM Classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#plt.style.use('bmh')
#plt.style.use('ggplot')
plt.style.use('seaborn-notebook')
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import keras
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,PReLU
from kerastuner.tuners import RandomSearch

In [3]:
#deep learning liabraries
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops

In [4]:

dataset=pd.read_csv('trainho.csv')
test_df=pd.read_csv('testho.csv')

## print shape of dataset with rows and columns
print(dataset.shape)

(1460, 81)


In [ ]:
test_df.shape

In [ ]:
features_with_na=[features for features in dataset.columns if dataset[features].isnull().sum()>1]
## 2- step print the feature name and the percentage of missing values

for feature in features_with_na:
    print(feature, np.round(dataset[feature].isnull().mean(), 4),  ' % missing values')

In [ ]:

for feature in features_with_na:
    data = dataset.copy()
    
    # let's make a variable that indicates 1 if the observation was missing or zero otherwise
    data[feature] = np.where(data[feature].isnull(), 1, 0)
    
    # let's calculate the mean SalePrice where the information is missing or present
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.title(feature)
    plt.show()


In [ ]:
print("Id of Houses {}".format(len(dataset.Id)))

In [ ]:
# list of numerical variables
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtypes != 'O']

print('Number of numerical variables: ', len(numerical_features))

# visualise the numerical variables
dataset[numerical_features].head()

In [ ]:
numerical_features_test = [feature for feature in test_df.columns if test_df[feature].dtypes != 'O']

print('Number of numerical variables: ', len(numerical_features))

# visualise the numerical variables
test_df[numerical_features_test].head()

In [ ]:
year_feature = [feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature]

year_feature

In [ ]:
year_feature_test = [feature for feature in numerical_features_test if 'Yr' in feature or 'Year' in feature]

year_feature

In [ ]:

## Lets analyze the Temporal Datetime Variables
## We will check whether there is a relation between year the house is sold and the sales price

dataset.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('Median House Price')
plt.title("House Price vs YearSold")

In [ ]:
for feature in year_feature:
    if feature!='YrSold':
        data=dataset.copy()
        ## We will capture the difference between year variable and year the house was sold for
        data[feature]=data['YrSold']-data[feature]

        plt.scatter(data[feature],data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.show()


In [ ]:
discrete_feature=[feature for feature in numerical_features if len(dataset[feature].unique())<25 and feature not in year_feature+['Id']]
print("Discrete Variables Count: {}".format(len(discrete_feature)))


In [ ]:
dataset['MiscVal'].value_counts()

In [ ]:
test_df['MiscVal'].value_counts()

In [ ]:
discrete_feature

In [ ]:
discrete_feature_test

In [ ]:
discrete_feature_test=[feature for feature in numerical_features_test if len(test_df[feature].unique())<25 and feature not in year_feature_test+['Id']]
print("Discrete Variables Count: {}".format(len(discrete_feature_test)))

In [ ]:
for feature in discrete_feature:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

In [ ]:
continuous_feature=[feature for feature in numerical_features if feature not in discrete_feature+year_feature+['Id']]
print("Continuous feature Count {}".format(len(continuous_feature)))

In [ ]:
for feature in continuous_feature:
    data=dataset.copy()
    data[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.title(feature)
    plt.show()

In [ ]:
for feature in continuous_feature:
    data=dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data['SalePrice']=np.log(data['SalePrice'])
        plt.scatter(data[feature],data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalesPrice')
        plt.title(feature)
        plt.show()

In [ ]:
for feature in continuous_feature:
    data=dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
categorical_features=[feature for feature in dataset.columns if data[feature].dtypes=='O']
len(categorical_features)

In [ ]:
categorical_features_test=[feature for feature in test_df.columns if test_df[feature].dtypes=='O']
len(categorical_features_test)

In [ ]:
for feature in categorical_features:
    print('The feature is {} and number of categories are {}'.format(feature,len(dataset[feature].unique())))


In [ ]:

for feature in categorical_features:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

# Missing values

In [5]:
features_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes=='O']

for feature in features_nan:
    print("{}: {}% missing values".format(feature,np.round(dataset[feature].isnull().mean(),4)))

Alley: 0.9377% missing values
MasVnrType: 0.0055% missing values
BsmtQual: 0.0253% missing values
BsmtCond: 0.0253% missing values
BsmtExposure: 0.026% missing values
BsmtFinType1: 0.0253% missing values
BsmtFinType2: 0.026% missing values
FireplaceQu: 0.4726% missing values
GarageType: 0.0555% missing values
GarageFinish: 0.0555% missing values
GarageQual: 0.0555% missing values
GarageCond: 0.0555% missing values
PoolQC: 0.9952% missing values
Fence: 0.8075% missing values
MiscFeature: 0.963% missing values


In [6]:
features_nan_test=[feature for feature in test_df.columns if test_df[feature].isnull().sum()>1 and test_df[feature].dtypes=='O']

for feature in features_nan_test:
    print("{}: {}% missing values".format(feature,np.round(test_df[feature].isnull().mean(),4)))

MSZoning: 0.0027% missing values
Alley: 0.9267% missing values
Utilities: 0.0014% missing values
MasVnrType: 0.011% missing values
BsmtQual: 0.0302% missing values
BsmtCond: 0.0308% missing values
BsmtExposure: 0.0302% missing values
BsmtFinType1: 0.0288% missing values
BsmtFinType2: 0.0288% missing values
Functional: 0.0014% missing values
FireplaceQu: 0.5003% missing values
GarageType: 0.0521% missing values
GarageFinish: 0.0535% missing values
GarageQual: 0.0535% missing values
GarageCond: 0.0535% missing values
PoolQC: 0.9979% missing values
Fence: 0.8012% missing values
MiscFeature: 0.965% missing values


In [7]:
def replace_cat_feature(dataset,features_nan):
    data=dataset.copy()
    data[features_nan]=data[features_nan].fillna('Missing')
    return data

dataset=replace_cat_feature(dataset,features_nan)

dataset[features_nan].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [8]:
def replace_cat_feature_test(dataset,features_nan):
    data=test_df.copy()
    data[features_nan_test]=data[features_nan_test].fillna('Missing')
    return data

test_df=replace_cat_feature_test(test_df,features_nan_test)

test_df[features_nan_test].isnull().sum()

MSZoning        0
Alley           0
Utilities       0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Functional      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [9]:
numerical_with_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(dataset[feature].isnull().mean(),4)))

LotFrontage: 0.1774% missing value
MasVnrArea: 0.0055% missing value
GarageYrBlt: 0.0555% missing value


In [10]:
numerical_with_nan_test=[feature for feature in test_df.columns if test_df[feature].isnull().sum()>1 and test_df[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan_test:
    print("{}: {}% missing value".format(feature,np.around(test_df[feature].isnull().mean(),4)))

LotFrontage: 0.1556% missing value
MasVnrArea: 0.0103% missing value
BsmtFullBath: 0.0014% missing value
BsmtHalfBath: 0.0014% missing value
GarageYrBlt: 0.0535% missing value


In [11]:
for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=dataset[feature].median()
    
    ## create a new feature to capture nan values
    dataset[feature+'nan']=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_value,inplace=True)
    
dataset[numerical_with_nan].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [12]:
for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=test_df[feature].median()
    
    ## create a new feature to capture nan values
    test_df[feature+'nan']=np.where(test_df[feature].isnull(),1,0)
    test_df[feature].fillna(median_value,inplace=True)
    
test_df[numerical_with_nan_test].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFullBath    2
BsmtHalfBath    2
GarageYrBlt     0
dtype: int64

In [13]:
xyz=test_df['BsmtFullBath'].median()
test_df['BsmtFullBath'].fillna(xyz,inplace=True)

yxz=test_df['BsmtHalfBath'].median()
test_df['BsmtHalfBath'].fillna(yxz,inplace=True)

In [14]:
test_df[numerical_with_nan_test].isnull().sum()

LotFrontage     0
MasVnrArea      0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
dtype: int64

In [15]:
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
       
    dataset[feature]=dataset['YrSold']-dataset[feature]
    test_df[feature]=test_df['YrSold']-test_df[feature]

In [16]:
test_df.shape

(1459, 83)

In [17]:
num_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

for feature in num_features:
    dataset[feature]=np.log(dataset[feature])
    test_df[feature]=np.log(test_df[feature])

In [18]:
dataset['SalePrice']=np.log(dataset['SalePrice'])

In [19]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']

In [20]:
categorical_features

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [21]:
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')

In [22]:
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    test_df[feature]=np.where(test_df[feature].isin(temp_df),test_df[feature],'Rare_var')

In [ ]:
dataset.head(100)


In [ ]:
test_df.head(100)

In [23]:

for feature in categorical_features:
    labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    dataset[feature]=dataset[feature].map(labels_ordered)
    test_df[feature]=test_df[feature].map(labels_ordered)

In [ ]:
c=dataset1.groupby(['MSZoning'])['SalePrice'].mean().sort_values().index
c={k:i for i,k in enumerate(c,0)}

In [24]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
0,1461,20,2,4.382027,9.360655,1,2,0,1,1,...,2,2,0,6,2010,2,3,0,0,0
1,1462,20,3,4.394449,9.565704,1,2,1,1,1,...,4,0,12500,6,2010,2,3,0,0,0
2,1463,60,3,4.304065,9.534595,1,2,1,1,1,...,2,2,0,3,2010,2,3,0,0,0
3,1464,60,3,4.356709,9.208138,1,2,1,1,1,...,4,2,0,6,2010,2,3,0,0,0
4,1465,120,3,3.761200,8.518193,1,2,1,3,1,...,4,2,0,1,2010,2,3,0,0,0


In [ ]:
dataset1=pd.read_csv('trainho.csv')

In [ ]:
labels_ordered

In [25]:
##feature scaling
scaling_feature=[feature for feature in dataset.columns if feature not in ['Id','SalePerice'] ]
len(scaling_feature)


83

In [26]:
feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])
scaler.fit(test_df[feature_scale])

MinMaxScaler()

In [ ]:
dataset.head()

In [ ]:
test_df.head()

In [27]:
data = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(dataset[feature_scale]), columns=feature_scale)],
                    axis=1)

In [29]:
data.columns

Index(['Id', 'SalePrice', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea',
       'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond'

In [30]:
test=test_df.copy()

In [31]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [32]:
y_train=dataset[['SalePrice']]

In [33]:
X_train=dataset.drop(['SalePrice','Id'],axis=1)

In [34]:
X_test=test_df.drop('Id',axis=1)

In [35]:
X_test.shape[1]==X_train.shape[1]

True

In [36]:
y_train

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216
...,...
1455,12.072541
1456,12.254863
1457,12.493130
1458,11.864462


In [55]:
feature_selection_Model=SelectFromModel(Lasso(alpha=0.006,random_state=0))
feature_selection_Model.fit(X_train,y_train)

SelectFromModel(estimator=Lasso(alpha=0.006, random_state=0))

In [ ]:
feature_selection_Model.get_support()

In [56]:
selected_feature=X_train.columns[(feature_selection_Model.get_support())]

In [ ]:
len(selected_feature)

In [57]:
X_train=X_train[selected_feature]

In [58]:
X_test=X_test[selected_feature]

In [ ]:
import xgboost

In [ ]:
from xgboost import XGBRegressor

In [37]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb



In [71]:
lasso = Lasso(alpha =0.0005)
lasso.fit(X_train,y_train)
lasso_predict=lasso.predict(X_test)


In [100]:
lasso_predict1=lasso.predict(X_train)


In [72]:
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9)
ENet.fit(X_train,y_train)
Epredict=ENet.predict(X_test)

In [101]:
Epredict1=ENet.predict(X_train)

In [73]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,max_depth=4,max_features='sqrt',min_samples_leaf=15, min_samples_split=10,loss='huber')
GBoost.fit(X_train,y_train)
Gpredict=GBoost.predict(X_test)

C:\Users\NEERAJ\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [102]:
Gpredict1=GBoost.predict(X_train)

In [75]:
model_xgb = xgb.XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
model_xgb.fit(X_train,y_train)
xgb_predict=model_xgb.predict(X_test)

[01:44:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [103]:
xgb_predict1=model_xgb.predict(X_train)

In [104]:
lasso_predict1=pd.DataFrame(lasso_predict1,columns=['LASSO'])
Epredict1=pd.DataFrame(Epredict1,columns=['ENet'])
Gpredict1=pd.DataFrame(Gpredict1,columns=['Gboost'])
xgb_predict1=pd.DataFrame(xgb_predict1,columns=['XGboost'])

In [76]:
learner=[lasso_predict,Epredict,Gpredict,xgb_predict]

In [81]:
lasso_predict=pd.DataFrame(lasso_predict,columns=['LASSO'])
    

In [82]:
Epredict=pd.DataFrame(Epredict,columns=['ENet'])

In [83]:
Gpredict=pd.DataFrame(Gpredict,columns=['Gboost'])

In [84]:
xgb_predict=pd.DataFrame(xgb_predict,columns=['XGboost'])

In [112]:
Final_predict_train=pd.concat([lasso_predict1,Epredict1,Gpredict1,xgb_predict1],axis=1)

In [ ]:
Final_predict=pd.concat([lasso_predict,Epredict,Gpredict,xgb_predict],axis=1)

In [87]:
Final_predict1=Final_predict.copy()

In [106]:
for feature in Final_predict_train.columns:
    Final_predict_train[feature]=np.exp(Final_predict_train[feature])

In [113]:
Final_predict_train

,LASSO,ENet,Gboost,XGboost
0,12.200919,12.200740,12.240053,12.224954
1,12.221277,12.221434,12.108185,12.122427
2,12.257745,12.257768,12.306732,12.262051
3,12.027035,12.027007,11.861639,11.930984
4,12.645342,12.645187,12.457966,12.582577
...,...,...,...,...
1455,12.071306,12.071205,12.072080,12.069096
1456,12.225079,12.225043,12.239238,12.208678
1457,12.452311,12.452516,12.493440,12.487906
1458,11.870032,11.870098,11.850901,11.855632


In [65]:
for features in X_test.columns:
    if X_test[features].isnull().sum()!=0:
        X_test[features].fillna(X_test[features].median(),inplace=True)

In [118]:
Mean1=Final_predict1.mean(axis=1)

In [114]:
lasso1=Lasso(alpha=0.0005)
lasso1.fit(Final_predict_train,y_train)

Lasso(alpha=0.0005)

In [115]:
stacked1=lasso1.predict(Final_predict1)

In [67]:
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9)
rmse_cv(ENet)

array([0.11359715, 0.14049208, 0.12540819, 0.11665335, 0.16402104])

In [68]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,max_depth=4,max_features='sqrt',min_samples_leaf=15, min_samples_split=10,loss='huber')
rmse_cv(GBoost)


C:\Users\NEERAJ\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\NEERAJ\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\NEERAJ\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\NEERAJ\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

array([0.11401838, 0.13963868, 0.12720284, 0.1165457 , 0.12647099])

In [69]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
rmse_cv(model_xgb)

[01:37:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:37:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:37:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

array([0.11024148, 0.13026422, 0.12478346, 0.11410659, 0.12722071])

# Taking average of all outputs from each algorithm

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
xgbo = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

In [ ]:
model_xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = model_xgb.predict(X_test)
y_final=np.exp(y_pred_xgb )

In [119]:
output = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': np.exp(Mean1)})
output.to_csv('househouse1234.csv', index=False)

In [ ]:
!pip install mlxtend  

In [ ]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

xgb = XGBRegressor()

rf = RandomForestRegressor()
ridge = Ridge()
lasso = Lasso()
svr = SVR(kernel='linear')